In [1]:
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from glob import glob

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, Dropout, Dense, Input, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# resize all images
IMAGE_SIZE = [256, 256]

train_path = '/kaggle/input/plantvillage-dataset/color'
test_path = '/kaggle/input/plantvillage-dataset/segmented'

In [4]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], 
                  weights=None, #'imagenet'
                  include_top=False)

In [5]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [6]:
x = Flatten()(resnet.output)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
prediction = Dense(38, activation='softmax')(x)

In [7]:
# creating model
model = Model(inputs = resnet.input, outputs = prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Using the Image Data Generator
train_datagen = ImageDataGenerator(rotation_range=25, 
                                   width_shift_range=0.1,
                                   height_shift_range=0.1, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode="nearest", 
                                   validation_split=0.1)

In [10]:
training_set = train_datagen.flow_from_directory('/kaggle/input/plantvillage-dataset/color',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training'
                                                )

Found 48888 images belonging to 38 classes.


In [11]:
validation_set = train_datagen.flow_from_directory('/kaggle/input/plantvillage-dataset/color',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            subset='validation',
                                           )

Found 5417 images belonging to 38 classes.


In [12]:
training_set.class_indices

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Blueberry___healthy': 4,
 'Cherry_(including_sour)___Powdery_mildew': 5,
 'Cherry_(including_sour)___healthy': 6,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
 'Corn_(maize)___Common_rust_': 8,
 'Corn_(maize)___Northern_Leaf_Blight': 9,
 'Corn_(maize)___healthy': 10,
 'Grape___Black_rot': 11,
 'Grape___Esca_(Black_Measles)': 12,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13,
 'Grape___healthy': 14,
 'Orange___Haunglongbing_(Citrus_greening)': 15,
 'Peach___Bacterial_spot': 16,
 'Peach___healthy': 17,
 'Pepper,_bell___Bacterial_spot': 18,
 'Pepper,_bell___healthy': 19,
 'Potato___Early_blight': 20,
 'Potato___Late_blight': 21,
 'Potato___healthy': 22,
 'Raspberry___healthy': 23,
 'Soybean___healthy': 24,
 'Squash___Powdery_mildew': 25,
 'Strawberry___Leaf_scorch': 26,
 'Strawberry___healthy': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Toma

In [13]:
detection_model = model.fit(training_set,
                                      validation_data=validation_set,
                                      epochs=15,
                                      steps_per_epoch=len(training_set),
                                      validation_steps=len(validation_set)
                                     )

Epoch 1/15
1528/1528 [==============================] - 1448s 915ms/step - loss: 2.5849 - accuracy: 0.5107 - val_loss: 6.2380 - val_accuracy: 0.1898
Epoch 2/15
1528/1528 [==============================] - 1094s 716ms/step - loss: 0.7004 - accuracy: 0.7807 - val_loss: 1.3209 - val_accuracy: 0.6469
Epoch 3/15
1528/1528 [==============================] - 1106s 724ms/step - loss: 0.4772 - accuracy: 0.8463 - val_loss: 2.2359 - val_accuracy: 0.6432
Epoch 4/15
1528/1528 [==============================] - 1123s 735ms/step - loss: 0.3927 - accuracy: 0.8745 - val_loss: 2.2023 - val_accuracy: 0.5448
Epoch 5/15
1528/1528 [==============================] - 1073s 702ms/step - loss: 0.3304 - accuracy: 0.8940 - val_loss: 1.2094 - val_accuracy: 0.7336
Epoch 6/15
1528/1528 [==============================] - 1095s 717ms/step - loss: 0.3000 - accuracy: 0.9048 - val_loss: 0.6856 - val_accuracy: 0.8466
Epoch 7/15
1528/1528 [==============================] - 1087s 711ms/step - loss: 0.2511 - accuracy: 0.9194

Increase the number of epoch to increase accuracy and add early stopping.

In [15]:
# saving the trained model
import pickle
pickle.dump(model,open("model.h5","wb"))

In [16]:
# loading saved model
model_loaded = pickle.load(open("model.h5","rb"))

Run the above cell to access the saved trained model.